In [1]:
import os
os.chdir("../") 



In [2]:
%pwd

'c:\\Users\\P52s\\Documents\\Python Project\\SpellX'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [4]:

from spellX.constants import *
from spellX.utils.common import read_yaml , create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [6]:
import os
import urllib.request as request
import zipfile
from spellX import logger
from spellX.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [23]:
import traceback
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    print(f"An error occurred: {e}")
    # You can also print the traceback for more information

    traceback.print_exc()


[2023-08-30 10:53:48,453: INFO: 3875739439: yaml file: config\config.yaml loaded successfully]
[2023-08-30 10:53:48,464: INFO: 3875739439: yaml file: params.yaml loaded successfully]
[2023-08-30 10:53:48,469: INFO: common: created directory at: artifacts]
[2023-08-30 10:53:48,474: INFO: common: created directory at: artifacts/data_ingestion]


[2023-08-30 10:53:50,857: INFO: 4205007308: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 2531880
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "8b6a1998e98f95404f14dd445fa4d0374bf9f5c3be8f9b4df41ca600c0460112"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BD70:E4CF:7770D5:7C3B25:64EF11A7
Accept-Ranges: bytes
Date: Wed, 30 Aug 2023 09:53:49 GMT
Via: 1.1 varnish
X-Served-By: cache-mad22034-MAD
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1693389229.433488,VS0,VE289
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 7e2475c34fbc8437b26577c98a970df0ec3102bc
Expires: Wed, 30 Aug 2023 09:58:49 GMT
Source-Age: 0

]
